
<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Классификация изображений</b></h3>


# Домашнее задание. Классификация изображений

Сегодня вам предстоить помочь телекомпании FOX в обработке их контента. Как вы знаете, сериал "Симпсоны" идет на телеэкранах более 25 лет, и за это время скопилось очень много видеоматериала. Персоонажи менялись вместе с изменяющимися графическими технологиями, и Гомер Симпсон-2018 не очень похож на Гомера Симпсона-1989. В этом задании вам необходимо классифицировать персонажей, проживающих в Спрингфилде. Думаю, нет смысла представлять каждого из них в отдельности.



В нашем тесте будет 991 картинка, для которых вам будет необходимо предсказать класс.

## Шаг 1. Установка зависимостей

#### Установим необходимые библиотеки и проверим доступность CUDA

In [1]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
#
# if it prints otherwise, then you need to enable GPU:
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [18]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from torchvision.transforms import v2

import torchsummary

from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline
from sklearn.metrics import f1_score

import torch.optim as optim
from torchvision import models
import random, numpy as np, os

# в sklearn не все гладко, чтобы в colab удобно выводить картинки
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)


 #### Определим константы, которые будем использовать в по ходу ноутбука

In [4]:
# разные режимы датасета
DATA_MODES = ['train', 'val', 'test']

# работаем на видеокарте
DEVICE = torch.device("cuda")
PATH = '/kaggle/input/journey-to-springfield1'
#определим директории с тренировочными и тестовыми файлами
TRAIN_DIR = Path(f'{PATH}/train') #Path('./data/train/')
TEST_DIR = Path(f'{PATH}/testset') #Path('./data/testset')

# параметры нормировки изображений по трем каналам перед подачей в модель
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]

# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = [224, 224]

## Шаг 2. Загрузка и обработка данных

#### Скачаем изображения по ссылке

Посмотрите на структуру файлов в папках train и testset.

В train лежат данные, которые мы будем использовать для обучения модели. Изображения персонажей разложены по папкам, которые названы по именам персонажей. Названия папок мы в дальнейшем будет использовать в качестве текстовых меток классов.

В testset находятся изображения, для которых вам надо будет сделать прогноз наиболее вероятного класса.


Для обращения к файлам сформируем списки имен файлов обучающей+валидационнной и тестовой выборок. Это полные имена, включающие путь к файлам.


In [5]:
train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

In [6]:
print(f"Train files: {len(train_val_files)}")
print(f"Test files: {len(test_files)}")

Train files: 20933
Test files: 991


Кодировать имена персонажей в числовые метки класса и обратно будем при помощи `LabelEncoder`.

Для train выборки сформируем список текстовых меток всех изображений - имя родительской директории, которая одновременно является и именем персонажа. Зададим числовые метки классов нашего энкодера при помощи метода `fit`.

Далее будем применять метод `transform` для преобразования текстовых меток в числовые, и метод `inverse_transform` для преобразования числовых меток в текстовые.


In [7]:
label_encoder = LabelEncoder()

train_val_labels = [path.parent.name for path in train_val_files]

label_encoder.fit(train_val_labels)

LabelEncoder()

Разделим train выборку на обучающую и валидационнную части. Для того, чтобы персонажи были пропорционально представлены в обучающей и валидационнной подвыборках, применим стратификацию по меткам класса.

In [8]:
from sklearn.model_selection import train_test_split

train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

#### Создадим Datasets и Dataloaders

Важно разобраться, что делает метод self.transform_images_to_tensors().

`Compose` объединяет последовательность следующих преобразований:
- `PILToTensor` конвертирует  `PIL Image` в тензор с параметрами в диапазоне $[0, 255]$ (как все пиксели в исходном изображении)
- `ToDtype` преобразует тензор в `FloatTensor` размера ($C \times H \times W$) со значениями пикселей в диапазоне $[0,1]$
- затем `Normalize` производится масштабирование:
$\text{input} = \frac{\text{input} - \text{mean}}{\text{std}} $, <br>      где константы mean и std - средние и дисперсии по каналам в датасете ImageNet
- наконец, `Resize` преобразует картинки к размеру $224 \times 224$ (в описании датасета указано, что картинки разного размера, так как брались напрямую с видео, поэтому следует привести их к одному размеру).

Сейчас аугментация не изображений не производится, поэтому для обучающих и валидационных/тестовых изображений производится одинаковая трансформация. В дальнейшем, если вы захотите добавить аугментацию, вы можете сделать это, например, модифицировав метод `transform_images_to_tensors`. Подробнее про трансформацию изображений вы можете почитать в документации: https://docs.pytorch.org/vision/main/transforms.html


In [9]:
class SimpsonsDataset(Dataset):
    def __init__(self, files, label_encoder, mode, augment_type='basic'):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode
        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.label_encoder = label_encoder
        self.len_ = len(self.files)

        self.augment_type = augment_type  # <- флажок для аугументации

    def __len__(self):
        return self.len_ # сейчас self.__len__() возвращает количество картинок, подаваемых на вход.
        # Если вы решите перевзвесить размеры категорий внутри класса -
        # не забудьте изменить вывод self.__len__()

    def __getitem__(self, index):
        x = self.load_image(self.files[index])
        x = self.transform_images_to_tensors(x)

        if self.mode == 'test':
            return x
        else:
            path = self.files[index]
            y = self.label_encoder.transform([path.parent.name,]).item()
            return x, y

    # принимает путь к файлу изображения и возвращает само изображение
    def load_image(self, file):
        image = Image.open(file)
        image.load()
        return image

    # преобразует изображение в тензор
    def transform_images_to_tensors(self, image):
      if self.mode == 'train':
        if self.augment_type == 'basic':
            transform = v2.Compose([
                v2.Resize(RESCALE_SIZE),
                v2.PILToTensor(),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
                
            ])
        elif self.augment_type == 'custom':
            transform = v2.Compose([
            v2.Resize(RESCALE_SIZE),
            v2.RandomHorizontalFlip(p=0.5),
            v2.RandomRotation(degrees=15),
            v2.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.1
            ),
            v2.PILToTensor(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
            ])
        
      else:
        transform = v2.Compose([
            v2.Resize(RESCALE_SIZE),
            v2.PILToTensor(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
            
          ])

      tensor_transformed = transform(image)
      return(tensor_transformed)


In [11]:
train_dataset = SimpsonsDataset(train_files, label_encoder = label_encoder, mode='train', augment_type='custom')
val_dataset = SimpsonsDataset(val_files, label_encoder, mode='val')

In [12]:
batch_size = 128

In [13]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
loaders = {'train':train_loader, 'val': val_loader}

Напишите функции:
- для обучения модели на одной эпохе
- для валидации модели на одной эпохе
- для реализации полного цикла обучения

За основу можно взять функции, которые вы написали в предыдущем домашнем задании.

In [14]:
from tqdm import tqdm
def train_one_epoch(model, dataloader, loss_func, optimizer, device):
    """
    Args:
        model: модель PyTorch
        dataloader: DataLoader с обучающей выборкой
        loss_func: функция потерь
        optimizer: оптимизатор
        device: 'cpu', 'cuda' или 'mps'
    
    Returns:
        epoch_loss, epoch_acc, epoch_f1, all_preds, all_labels
    """
    model.train()
    all_preds = []
    all_labels = []
    running_loss = 0.0

    for images, labels in tqdm(dataloader, desc='Training', leave=False):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        preds = torch.argmax(outputs, dim=1)
        all_preds.append(preds.cpu())
        all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    epoch_loss = running_loss / len(all_labels)
    epoch_acc = (all_preds == all_labels).sum().item() / len(all_labels)
    epoch_f1 = f1_score(all_labels.numpy(), all_preds.numpy(), average='micro')

    return epoch_loss, epoch_acc, epoch_f1



In [15]:
def val_one_epoch(model, dataloader, loss_func, device):
    """
    Args:
        model: модель PyTorch
        dataloader: DataLoader с валидационной выборкой
        loss_func: функция потерь
        device: 'cpu', 'cuda' или 'mps'
    
    Returns:
        epoch_loss, epoch_acc, epoch_f1, all_preds, all_labels
    """
    model.eval()
    all_preds = []
    all_labels = []
    running_loss = 0.0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc='Validation', leave=False):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = loss_func(outputs, labels)
            running_loss += loss.item() * images.size(0)

            preds = torch.argmax(outputs, dim=1)
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    epoch_loss = running_loss / len(all_labels)
    epoch_acc = (all_preds == all_labels).sum().item() / len(all_labels)
    epoch_f1 = f1_score(all_labels.numpy(), all_preds.numpy(), average='micro')

    return epoch_loss, epoch_acc, epoch_f1

In [16]:
def train_model(model, train_loader, val_loader, loss_func, optimizer, num_epochs=10, device=None):
    """
    Args:
        model: PyTorch модель
        train_loader: DataLoader для обучения
        val_loader: DataLoader для валидации
        loss_func: функция потерь
        optimizer: оптимизатор
        num_epochs: количество эпох
        device: 'cpu', 'cuda' или 'mps'. Если None, выбирается автоматически.
    
    Returns:
        model: обученная модель
        history: словарь с историей метрик
    """

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else 
                              "mps" if torch.backends.mps.is_available() else "cpu")
    print(f"Training on device: {device}")
    
    model.to(device)

    best_f1 = 0.0
    history = {"train_loss": [], "train_acc": [], "train_f1": [],
               "val_loss": [], "val_acc": [], "val_f1": []}

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")

        # Тренировка 
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, loss_func, optimizer, device)
        
        #Валидация
        val_loss, val_acc, val_f1 = val_one_epoch(model, val_loader, loss_func, device)

        # Вывод
        print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | F1: {train_f1:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}")

        #Сохранение истории
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["train_f1"].append(train_f1)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)
        history["val_f1"].append(val_f1)

        #Сохраняем лучшую модель по F1
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")

    print(f"\nBest Validation F1: {best_f1:.4f}")
    return model, history


In [ ]:
вобще можно было выводить только accuracy или f1-score(micro) - т.к. это одно и тоже, но пусть будет) 

## Шаг 6. Submit на Kaggle

Создадим loader для тестовых данных

In [26]:
test_dataset = SimpsonsDataset(test_files, label_encoder = label_encoder, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)

Воспользуемся функцией predict, которая возвращает предсказанные числовые метки для всех объектов в лоадере.

In [27]:
def predict(model, loader):
    model.eval()
    all_predictions = torch.tensor([]).to(DEVICE).int()
    print("Test mode...")
    for inputs in tqdm_notebook(loader):
        inputs = inputs.to(DEVICE)

        with torch.no_grad():
            outputs = model(inputs)

            predictions = outputs.argmax(-1).int()
            all_predictions = torch.cat((all_predictions, predictions), 0)
    return all_predictions.cpu()

Получем предсказание меток классов для тестовых данных:

In [29]:
predicted_numeric_labels = predict(model, test_loader)

Test mode...


  0%|          | 0/16 [00:00<?, ?it/s]

и преобразуем их в текстовые метки:

In [30]:
predicted_text_labels = label_encoder.inverse_transform(predicted_numeric_labels)

Загрузим пример файла для загрузки на Kaggle (проверьте путь, по которому у вас лежит файл sample_submission.csv и при необходимости скорректируйте путь в коде ниже):

In [34]:
import pandas as pd
sample_submission = pd.read_csv("/kaggle/input/journey-to-springfield1/sample_submission.csv")
sample_submission.head(10)

,Id,Expected
0,img0.jpg,bart_simpson
1,img1.jpg,bart_simpson
2,img2.jpg,bart_simpson
3,img3.jpg,bart_simpson
4,img4.jpg,bart_simpson
5,img5.jpg,bart_simpson
6,img6.jpg,bart_simpson
7,img7.jpg,bart_simpson
8,img8.jpg,bart_simpson
9,img9.jpg,bart_simpson


In [35]:
my_submission = pd.DataFrame({'Id': [path.name for path in test_files], 'Expected': predicted_text_labels})
my_submission.head(10)

,Id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,bart_simpson
2,img10.jpg,ned_flanders
3,img100.jpg,chief_wiggum
4,img101.jpg,apu_nahasapeemapetilon
5,img102.jpg,kent_brockman
6,img103.jpg,edna_krabappel
7,img104.jpg,chief_wiggum
8,img105.jpg,lisa_simpson
9,img106.jpg,kent_brockman


In [38]:
my_submission.to_csv('/kaggle/working/mixup50.csv', index=False)

## Собственные эксперементы:

### Работа с даннымм 

У нас явно присутсвует дизбаланс в обучающейся выборке, посмотрим какой и потом попробуем его исправить

In [84]:
from collections import Counter

# достаём имена папок 
class_names = [path.parent.name for path in train_dataset.files]

# считаем количество изображений каждого класса
class_counts = Counter(class_names)

# сортируем для наглядности
for cls, count in sorted(class_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{cls}: {count}")


homer_simpson: 1684
ned_flanders: 1090
moe_szyslak: 1089
lisa_simpson: 1015
bart_simpson: 1006
marge_simpson: 968
krusty_the_clown: 904
charles_montgomery_burns: 895
principal_skinner: 895
milhouse_van_houten: 809
chief_wiggum: 739
abraham_grampa_simpson: 685
sideshow_bob: 658
apu_nahasapeemapetilon: 467
kent_brockman: 373
comic_book_guy: 352
edna_krabappel: 343
nelson_muntz: 269
lenny_leonard: 233
mayor_quimby: 185
waylon_smithers: 136
maggie_simpson: 96
groundskeeper_willie: 91
barney_gumble: 80
selma_bouvier: 77
carl_carlson: 74
ralph_wiggum: 67
patty_bouvier: 54
martin_prince: 53
professor_john_frink: 49
snake_jailbird: 41
cletus_spuckler: 35
rainier_wolfcastle: 34
agnes_skinner: 32
sideshow_mel: 30
otto_mann: 24
fat_tony: 20
gil: 20
miss_hoover: 13
disco_stu: 6
troy_mcclure: 6
lionel_hutz: 2


In [22]:

# Устройство 
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Device:", device)


num_classes = len(label_encoder.classes_)

# ResNet50 без загрузки предобученных весов 
model = models.resnet50(weights=None)  
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

#Оптимизатор
lr_head = 1e-3
lr_backbone = 1e-4
weight_decay = 1e-4

backbone_params = []
head_params = []
for name, p in model.named_parameters():
    if name.startswith("fc."):
        head_params.append(p)
    else:
        backbone_params.append(p)

optimizer = torch.optim.AdamW([
    {"params": backbone_params, "lr": lr_backbone},
    {"params": head_params, "lr": lr_head}
], weight_decay=weight_decay)


criterion = nn.CrossEntropyLoss()

num_epochs = 10
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

Обучение
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())



model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_func=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs
)


Device: cuda


/tmp/ipykernel_48/1895605242.py:48: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


Training on device: cuda

Epoch 1/10


Train Loss: 3.1528 | Acc: 0.1410 | F1: 0.1410
Val   Loss: 2.7414 | Acc: 0.2499 | F1: 0.2499

Epoch 2/10


Train Loss: 2.1710 | Acc: 0.4124 | F1: 0.4124
Val   Loss: 1.7581 | Acc: 0.5453 | F1: 0.5453

Epoch 3/10


...

Train Loss: 0.6285 | Acc: 0.8235 | F1: 0.8235
Val   Loss: 0.7698 | Acc: 0.7889 | F1: 0.7889

Epoch 7/10


Train Loss: 0.5334 | Acc: 0.8484 | F1: 0.8484
Val   Loss: 0.7286 | Acc: 0.8013 | F1: 0.8013

Epoch 8/10


Train Loss: 0.4555 | Acc: 0.8691 | F1: 0.8691
Val   Loss: 0.5153 | Acc: 0.8594 | F1: 0.8594

Epoch 9/10


Train Loss: 0.4085 | Acc: 0.8834 | F1: 0.8834
Val   Loss: 0.4906 | Acc: 0.8716 | F1: 0.8716

Epoch 10/10


Train Loss: 0.3608 | Acc: 0.8958 | F1: 0.8958
Val   Loss: 0.4803 | Acc: 0.8752 | F1: 0.8752

Best Validation F1: 0.8752


Можно заметить, что как будто не хватило количесвто эпох, но даже если увеличить, не получится выбить скор 0.97+

In [18]:
train_dataset = SimpsonsDataset(train_files, label_encoder = label_encoder, mode='train', augment_type='custom') 
val_dataset = SimpsonsDataset(val_files, label_encoder, mode='val')
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
loaders = {'train':train_loader, 'val': val_loader}

augment_type='custom' - добавление агументации 

In [28]:

# Параметры

SEED = 42
num_epochs = 12
lr_head = 1e-3
lr_backbone = 1e-4
weight_decay = 1e-4
model_path = "best_resnet50.pth"

# Воспроизводимость и устройство
def seed_everything(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else 
                      "mps" if torch.backends.mps.is_available() else "cpu")
print("Device:", device)


# Модель с более сильным класификатором 

def build_resnet50(num_classes, pretrained=True, dropout_p=0.4):
    model = models.resnet50(pretrained=pretrained)
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(inplace=True),
        nn.Dropout(dropout_p),
        nn.Linear(1024, num_classes)
    )
    return model

num_classes = len(label_encoder.classes_)
model = build_resnet50(num_classes=num_classes, pretrained=True)
model = model.to(device)

# Оптимизатор
backbone_params = [p for n,p in model.named_parameters() if not n.startswith("fc.")]
head_params = [p for n,p in model.named_parameters() if n.startswith("fc.")]

optimizer = optim.AdamW([
    {"params": backbone_params, "lr": lr_backbone},
    {"params": head_params, "lr": lr_head}
], weight_decay=weight_decay)

criterion = nn.CrossEntropyLoss()


#  Заморозка backbone 
for name, param in model.named_parameters():
    if not name.startswith("fc."):
        param.requires_grad = False


# Запуск обучения 
model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_func=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device
)

print("Training finished. Best model saved as 'best_model.pth'.")


Device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training on device: cuda

Epoch 1/12


Train Loss: 1.4393 | Acc: 0.6217 | F1: 0.6217
Val   Loss: 1.1834 | Acc: 0.6660 | F1: 0.6660

Epoch 2/12


Train Loss: 0.9727 | Acc: 0.7291 | F1: 0.7291
Val   Loss: 1.0421 | Acc: 0.7060 | F1: 0.7060

Epoch 3/12


...

Train Loss: 0.5982 | Acc: 0.8210 | F1: 0.8210
Val   Loss: 0.8605 | Acc: 0.7606 | F1: 0.7606

Epoch 9/12


Train Loss: 0.5439 | Acc: 0.8366 | F1: 0.8366
Val   Loss: 0.8304 | Acc: 0.7719 | F1: 0.7719

Epoch 10/12


Train Loss: 0.5245 | Acc: 0.8402 | F1: 0.8402
Val   Loss: 0.7774 | Acc: 0.7803 | F1: 0.7803

Epoch 11/12


Train Loss: 0.5143 | Acc: 0.8439 | F1: 0.8439
Val   Loss: 0.7490 | Acc: 0.7925 | F1: 0.7925

Epoch 12/12


Train Loss: 0.4811 | Acc: 0.8541 | F1: 0.8541
Val   Loss: 0.7884 | Acc: 0.7828 | F1: 0.7828

Best Validation F1: 0.7925
Training finished. Best model saved as 'best_model.pth'.


Скор получился ещё хуже( 

In [19]:

num_classes = len(label_encoder.classes_)
model = build_resnet50(num_classes=num_classes, pretrained=True)
model = model.to(device)

backbone_params = [p for n,p in model.named_parameters() if not n.startswith("fc.")]
head_params = [p for n,p in model.named_parameters() if n.startswith("fc.")]
optimizer = optim.AdamW([
    {"params": backbone_params, "lr": lr_backbone},
    {"params": head_params, "lr": lr_head}
], weight_decay=weight_decay)

criterion = nn.CrossEntropyLoss()

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_func=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device
)

torch.save(model.state_dict(), model_path)
print("Training finished. Best model saved as", model_path)


Device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training on device: cuda

Epoch 1/12


Train Loss: 0.5967 | Acc: 0.8557 | F1: 0.8557
Val   Loss: 0.2294 | Acc: 0.9431 | F1: 0.9431

Epoch 2/12


Train Loss: 0.1268 | Acc: 0.9683 | F1: 0.9683
Val   Loss: 0.1579 | Acc: 0.9606 | F1: 0.9606

Epoch 3/12


...

Train Loss: 0.0248 | Acc: 0.9927 | F1: 0.9927
Val   Loss: 0.1740 | Acc: 0.9656 | F1: 0.9656

Epoch 9/12


Train Loss: 0.0352 | Acc: 0.9896 | F1: 0.9896
Val   Loss: 0.1721 | Acc: 0.9648 | F1: 0.9648

Epoch 10/12


Train Loss: 0.0267 | Acc: 0.9925 | F1: 0.9925
Val   Loss: 0.1625 | Acc: 0.9671 | F1: 0.9671

Epoch 11/12


Train Loss: 0.0171 | Acc: 0.9954 | F1: 0.9954
Val   Loss: 0.1632 | Acc: 0.9669 | F1: 0.9669

Epoch 12/12


Train Loss: 0.0181 | Acc: 0.9946 | F1: 0.9946
Val   Loss: 0.1678 | Acc: 0.9664 | F1: 0.9664

Best Validation F1: 0.9679
Training finished. Best model saved as best_resnet50.pth


Ну вот, на конец-то хорошие результаты, продолжим эксперементы 
Попробую всё тоже самое, только не используя агументацию 

In [23]:
train_dataset = SimpsonsDataset(train_files, label_encoder = label_encoder, mode='train', augment_type='custom')
val_dataset = SimpsonsDataset(val_files, label_encoder, mode='val')
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
loaders = {'train':train_loader, 'val': val_loader}

In [22]:
torch.cuda.empty_cache() #очищаем кэш

#Пересоздаём модель и оптимизатор
model = build_resnet50(num_classes=num_classes, pretrained=True)
model = model.to(device)

backbone_params = [p for n,p in model.named_parameters() if not n.startswith("fc.")]
head_params = [p for n,p in model.named_parameters() if n.startswith("fc.")]
optimizer = optim.AdamW([
    {"params": backbone_params, "lr": lr_backbone},
    {"params": head_params, "lr": lr_head}
], weight_decay=weight_decay)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_func=criterion,
    optimizer=optimizer,
    num_epochs=7,
    device=device
)

torch.save(model.state_dict(), model_path)
print("Training finished. Best model saved as", model_path)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training on device: cuda

Epoch 1/7


Train Loss: 0.5268 | Acc: 0.8753 | F1: 0.8753
Val   Loss: 0.1927 | Acc: 0.9559 | F1: 0.9559

Epoch 2/7


Train Loss: 0.0609 | Acc: 0.9855 | F1: 0.9855
Val   Loss: 0.1586 | Acc: 0.9616 | F1: 0.9616

Epoch 3/7


Train Loss: 0.0138 | Acc: 0.9970 | F1: 0.9970
Val   Loss: 0.1500 | Acc: 0.9643 | F1: 0.9643

Epoch 4/7


Train Loss: 0.0056 | Acc: 0.9989 | F1: 0.9989
Val   Loss: 0.1393 | Acc: 0.9696 | F1: 0.9696

Epoch 5/7


Train Loss: 0.0050 | Acc: 0.9989 | F1: 0.9989
Val   Loss: 0.1487 | Acc: 0.9689 | F1: 0.9689

Epoch 6/7


Train Loss: 0.0159 | Acc: 0.9956 | F1: 0.9956
Val   Loss: 0.2178 | Acc: 0.9480 | F1: 0.9480

Epoch 7/7


Train Loss: 0.0372 | Acc: 0.9899 | F1: 0.9899
Val   Loss: 0.2153 | Acc: 0.9509 | F1: 0.9509

Best Validation F1: 0.9696
Training finished. Best model saved as best_resnet50.pth


Аугментация всё таки приносила свои плоды, так что вернём её и попробуем метод mixup augmentation

Но для этого надо немного изменить функции обучения

In [24]:
def mixup_data(x, y, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def train_one_epoch(model, dataloader, loss_func, optimizer, device, use_mixup=False, alpha=0.4):
    model.train()
    total_loss, total_correct, n = 0, 0, 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        if use_mixup:
            mixed_x, y_a, y_b, lam = mixup_data(images, labels, alpha)
            outputs = model(mixed_x)
            loss = mixup_criterion(loss_func, outputs, y_a, y_b, lam)
        else:
            outputs = model(images)
            loss = loss_func(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
        if not use_mixup:
            preds = outputs.argmax(dim=1)
            total_correct += (preds == labels).sum().item()
            n += labels.size(0)

    avg_loss = total_loss / len(dataloader.dataset)
    avg_acc = total_correct / n if n > 0 else 0.0
    return avg_loss, avg_acc

def evaluate_model(model, dataloader, loss_func, device):
    model.eval()
    total_loss, total_correct, n = 0, 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_func(outputs, labels)
            total_loss += loss.item() * labels.size(0)
            preds = outputs.argmax(dim=1)
            total_correct += (preds == labels).sum().item()
            n += labels.size(0)
    avg_loss = total_loss / len(dataloader.dataset)
    avg_acc = total_correct / n
    return avg_loss, avg_acc

def train_model(model, train_loader, val_loader, loss_func, optimizer, num_epochs, device, scheduler=None, use_mixup=False):
    best_val_acc = 0
    history = {"train_acc": [], "val_acc": []}

    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, loss_func, optimizer, device, use_mixup=use_mixup)
        val_loss, val_acc = evaluate_model(model, val_loader, loss_func, device)

        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        if scheduler:
            scheduler.step()

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_mixup_model.pth")

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}\n")

    return model, history


In [25]:
torch.cuda.empty_cache() #очищаем кэш

#Пересоздаём модель и оптимизатор
model = build_resnet50(num_classes=num_classes, pretrained=True)
model = model.to(device)

backbone_params = [p for n,p in model.named_parameters() if not n.startswith("fc.")]
head_params = [p for n,p in model.named_parameters() if n.startswith("fc.")]
optimizer = optim.AdamW([
    {"params": backbone_params, "lr": lr_backbone},
    {"params": head_params, "lr": lr_head}
], weight_decay=weight_decay)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_func=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    scheduler=scheduler,
    use_mixup=True   
)

torch.save(model.state_dict(), model_path)
print("Training finished. Best model saved as", model_path)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/12
Train Loss: 1.3616 | Acc: 0.0000
Val   Loss: 0.4134 | Acc: 0.9280

Epoch 2/12
Train Loss: 0.9398 | Acc: 0.0000
Val   Loss: 0.3386 | Acc: 0.9463

Epoch 3/12
Train Loss: 0.9250 | Acc: 0.0000
Val   Loss: 0.3251 | Acc: 0.9608

Epoch 4/12
Train Loss: 0.8001 | Acc: 0.0000
Val   Loss: 0.2270 | Acc: 0.9706

Epoch 5/12
Train Loss: 0.7710 | Acc: 0.0000
Val   Loss: 0.2010 | Acc: 0.9710

Epoch 6/12
Train Loss: 0.7609 | Acc: 0.0000
Val   Loss: 0.1763 | Acc: 0.9771

Epoch 7/12
Train Loss: 0.7583 | Acc: 0.0000
Val   Loss: 0.1818 | Acc: 0.9754

Epoch 8/12
Train Loss: 0.7043 | Acc: 0.0000
Val   Loss: 0.1686 | Acc: 0.9782

Epoch 9/12
Train Loss: 0.6254 | Acc: 0.0000
Val   Loss: 0.1353 | Acc: 0.9794

Epoch 10/12
Train Loss: 0.5963 | Acc: 0.0000
Val   Loss: 0.1442 | Acc: 0.9803

Epoch 11/12
Train Loss: 0.6450 | Acc: 0.0000
Val   Loss: 0.1545 | Acc: 0.9801

Epoch 12/12
Train Loss: 0.6309 | Acc: 0.0000
Val   Loss: 0.1226 | Acc: 0.9815

Training finished. Best model saved as best_resnet50.pth


Точность на валидации немного выше, а вот на лидерборде такая же( - 0.99468


Теперь попробуем настоящую тяжёлую артилерию - ResNet152, если kaggle позволит)

Сразу не позвол, и после перезапуска( стирание всех возможных кешей и загрухок, тоже)  
Попытка 3 тоже не увенчалась успехом( Провал, ничего не помогло, поробуем ResNet101

Эх на ResNet101 тоже памяти не хватило

В общем, моделек я попробовал куда больше, оставил только те, что прибавляли точность, а вот нервов потратил ещё больше на всякие либы для обучения зависимости и просто миллионы ошибо.

Лучший скор показала модель resnet50 с аугментацией метод mixup на каких-то фотографиях показал себя лучше, а где то и хуже, судя из того, что скор остался таким же, хотя возможно всё дело в скрытой части лидербодра(5%)

Ник на кагле - Михаил_Звягинцев_414906596